In [1]:
#Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import no_update

In [2]:
# #Import required libraries
# import pandas as pd
# import dash
# import dash_html_components as html
# import dash_core_components as dcc
# from dash.dependencies import Input, Output
# import plotly.express as px
# from jupyter_dash import JupyterDash
# from dash import no_update

#Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

#spacex_df['class']=spacex_df['class'].replace([0,1],[1,0])

#Create a dash application
app = JupyterDash(__name__)
#JupyterDash.infer_jupyter_proxy_config()

#Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site_dropdown',
                                             options=[
                                                 {'label': 'All Sites', 'value': 'ALL'},
                                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                             ],
                                             value='ALL',
                                             placeholder="Select a Launch Site here",
                                             searchable=True),
                                html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload_slider',min=0, max=10000, step=1,
                                                marks={0: '0',2500: '2500',5000:'5000',7500:'7500',10000:'10000'},
                                                value=[min_payload,max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success_payload_scatter_chart')),
                                ])
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),Input(component_id='site_dropdown', component_property='value'))

def get_graph(site_dropdown):
    if site_dropdown == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='success-pie-chart')
        return fig
    else:
        # dataframe and pie_chart when site_dropdown is chosen
        site_df=spacex_df.loc[spacex_df['Launch Site']==site_dropdown]
        site_piechart = px.pie(site_df,names='class',title='Launchsite launch')
        return site_piechart
    
@app.callback(Output(component_id='success_payload_scatter_chart', component_property='figure'),[Input(component_id='site_dropdown', component_property='value'),Input(component_id='payload_slider', component_property='value')])

def update_graph(site_dropdown, payload_slider):
    if site_dropdown == 'ALL':
        filtered_data = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0])&(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        #fig2 = px.scatter(spacex_df, x="Payload Mass (kg)", y="class", color="Booster Version Category")
        scatterplot = px.scatter(filtered_data, x="Payload Mass (kg)", y="class", color="Booster Version Category")
        return scatterplot
    else:
        specific_df=spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        filtered_data = specific_df[(specific_df['Payload Mass (kg)']>=payload_slider[0])&(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        scplot = px.scatter(filtered_data, x="Payload Mass (kg)", y="class",color="Booster Version Category")
        return scplot


# Run the app
if __name__ == '__main__':
    app.run_server()


Dash app running on http://127.0.0.1:8050/


<ipython-input-2-e4982554a822>:76: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

